<a href="https://colab.research.google.com/github/sai1653/ibm-data-science/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from datetime import datetime
import yfinance as yf

In [2]:


def get_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period="1y")
        df = df.reset_index()
        df = df[['Date', 'Close']]
        df['Date'] = pd.to_datetime(df['Date'])
        return df
    except Exception as e:
        print(f"Error fetching stock data for {symbol}: {e}")
        return pd.DataFrame(columns=['Date', 'Close'])

def scrape_revenue_data(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find('table', {'class': 'historical_data_table'})
        if table is None:
            raise ValueError("Could not find the revenue table on the webpage")

        data = []
        for row in table.find_all('tr')[1:]:  # Skip header row
            cols = row.find_all('td')
            if len(cols) >= 2:
                date = cols[0].text.strip()
                revenue = cols[1].text.strip().replace('$', '').replace(',', '')
                if revenue:
                    data.append({'Date': date, 'Revenue': float(revenue)})

        df = pd.DataFrame(data)
        df['Date'] = pd.to_datetime(df['Date'])
        return df
    except Exception as e:
        print(f"Error scraping revenue data from {url}: {e}")
        return pd.DataFrame(columns=['Date', 'Revenue'])

def create_dashboard(stock_data, revenue_data, company):
    if stock_data.empty or revenue_data.empty:
        print(f"Not enough data to create dashboard for {company}")
        return

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

    ax1.plot(stock_data['Date'], stock_data['Close'])
    ax1.set_title(f'{company} Stock Price')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Stock Price')

    ax2.plot(revenue_data['Date'], revenue_data['Revenue'])
    ax2.set_title(f'{company} Revenue')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Revenue')

    plt.tight_layout()
    plt.savefig(f'{company}_dashboard.png')
    plt.close()
    print(f"Dashboard created for {company}")



In [3]:
# Tesla Analysis
tesla_stock = get_stock_data('TSLA')
tesla_revenue = scrape_revenue_data('https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue')
create_dashboard(tesla_stock, tesla_revenue, 'Tesla')



Dashboard created for Tesla


In [4]:
# GameStop Analysis
gme_stock = get_stock_data('GME')
gme_revenue = scrape_revenue_data('https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue')
create_dashboard(gme_stock, gme_revenue, 'GameStop')

print("Analysis complete!")

Dashboard created for GameStop
Analysis complete!


In [5]:
# Tesla Data Display
print("Tesla Stock Data - First 5 rows:")
tesla_stock = tesla_stock.reset_index(drop=True)
print(tesla_stock.head())
print("\nTesla Revenue Data - Last 5 rows:")
print(tesla_revenue.tail())



Tesla Stock Data - First 5 rows:
                       Date       Close
0 2023-08-03 00:00:00-04:00  259.320007
1 2023-08-04 00:00:00-04:00  253.860001
2 2023-08-07 00:00:00-04:00  251.449997
3 2023-08-08 00:00:00-04:00  249.699997
4 2023-08-09 00:00:00-04:00  242.190002

Tesla Revenue Data - Last 5 rows:
         Date  Revenue
10 2013-01-01   2013.0
11 2012-01-01    413.0
12 2011-01-01    204.0
13 2010-01-01    117.0
14 2009-01-01    112.0


In [6]:

print("\nGameStop Stock Data - First 5 rows:")
gme_stock = gme_stock.reset_index(drop=True)
print(gme_stock.head())
print("\nGameStop Revenue Data - Last 5 rows:")
print(gme_revenue.tail())




GameStop Stock Data - First 5 rows:
                       Date  Close
0 2023-08-03 00:00:00-04:00  20.93
1 2023-08-04 00:00:00-04:00  20.98
2 2023-08-07 00:00:00-04:00  21.07
3 2023-08-08 00:00:00-04:00  20.74
4 2023-08-09 00:00:00-04:00  19.93

GameStop Revenue Data - Last 5 rows:
         Date  Revenue
11 2013-01-01   8887.0
12 2012-01-01   9551.0
13 2011-01-01   9474.0
14 2010-01-01   9078.0
15 2009-01-01   8806.0


In [7]:

tesla_stock.to_csv('tesla_stock_data.csv', index=False)
tesla_revenue.to_csv('tesla_revenue_data.csv', index=False)
gme_stock.to_csv('gme_stock_data.csv', index=False)
gme_revenue.to_csv('gme_revenue_data.csv', index=False)

print("\nData has been displayed and saved to CSV files.")


Data has been displayed and saved to CSV files.
